In [28]:
import numpy as np

def rosenbrock(X,funcParams, doGradient):
  """
  Compute the Rosenbrock function and optionally its gradient.
  
  The Rosenbrock function is defined as:
      f(x, y) = (a - x)^2 + b * (y - x^2)^2
  where 'a' and 'b' are parameters that control the shape of the function.

  Parameters:
      X (array-like): A 2-element array representing the point [x, y] in 2D space
                      where the function and gradient are evaluated.
      funcParams (array-like): A 2-element array containing the parameters [a, b]
                                for the Rosenbrock function.
      doGradient (bool): If True, the function also returns the gradient vector.
                          If False, only the function value is returned.

  Returns:
      f (float): The value of the Rosenbrock function at the given point [x, y].
      gradf (numpy.ndarray or None): The gradient vector [df/dx, df/dy] if doGradient is True.
                                      If doGradient is False, returns None for the gradient.
  """
  x = X[0]
  y = X[1]
  
  a = funcParams[0]
  b = funcParams[1]
  
  x2 = x*x
  
  t1 = a-x
  t2 = y-x2
  
  f = t1**2 + b*t2**2
  
  if doGradient:
    dfdx = - 2*t1 - 4*b*x*t2
    dfdy = 2*b*t2
  
    gradf = np.array([dfdx,dfdy])
  
    return f, gradf
  else:
    return f, None

def line_search(X, S, func, funcParams, maxIter, tol):
    """
    Perform a golden section line search to find the optimal alpha that minimizes
    the function F(X + alpha * S).
    
    Parameters:
        X (array-like): Current point in the space.
        S (array-like): Search direction.
        tol (float): Tolerance for stopping criterion.
        max_iter (int): Maximum number of iterations.

    Returns:
        alpha_star (float): The optimal step size.
    """
    # Golden ratio
    phi = (1 + np.sqrt(5)) / 2
    inv_phi = 1 / phi

    # Initial interval [a, b]
    a = 0
    b = 1
    # Evaluate points within the interval
    alpha1 = b - inv_phi * (b - a)
    alpha2 = a + inv_phi * (b - a)

    # Compute objective function values at the points
    f1 = func(X + alpha1 * S, funcParams, False)
    f2 = func(X + alpha2 * S, funcParams, False)

    # Iteratively narrow the search interval
    for _ in range(maxIter):
        if abs(b - a) < tol:
            break

        # Compare function values and update the interval
        if f1 < f2:
            b = alpha2
            alpha2 = alpha1
            f2 = f1
            alpha1 = b - inv_phi * (b - a)
            f1 = func(X + alpha1 * S, funcParams, False)
        else:
            a = alpha1
            alpha1 = alpha2
            f1 = f2
            alpha2 = a + inv_phi * (b - a)
            f2 = func(X + alpha2 * S, funcParams, False)

    # Optimal step size is the midpoint of the final interval
    alpha_star = (a + b) / 2
    return alpha_star


def opt(X0,func, funcParams, maxLineSearchIters, lineSearchTol, absTol, relTol):
  # Step 1: Initialize
  X = X0
  
  iters = 0
  
  f, gradf = func(X, funcParams, True)
  a = np.dot(gradf, gradf)
  print("iter= ", iters, "x = ", X[0], "y = ", X[1], "f=", f)

  while True:

    # If first time or if Fletcher-Reeves fails, then just select steepest decent
    S = -gradf
    
    # Step 2: Line search to find optimal alpha
    alpha_star = line_search(X, S, func, funcParams, maxLineSearchIters, lineSearchTol)

    # Step 3: Check if alpha_star equals zero
    if np.abs(alpha_star)<=1e-10:
      print("Alpha is zero, algorithm exits.")
      break
    
    # Step 4: Update X
    X = X + alpha_star * S
    
    # Step 5: compute function and gradient at new X
    f, gradf = func(X, funcParams, True)

    while True:
      
      # Compute Fletcher-Reeves conjugate direction Update:
      b = np.dot(gradf,gradf)
      beta = b/a
      S = -gradf + beta*S
      a = b
      
      # if slope is greater than 0, break
      if np.dot(S,gradf)>=0:
        break
      
      
      alpha_star = line_search(X, S, func, funcParams, maxLineSearchIters, lineSearchTol)
      
      X_new = X + alpha_star * S
      f_new, gradf_new = func(X_new, funcParams, True)

      if np.abs(f_new - f)/np.abs(f)<= relTol or np.abs(f_new - f)<=absTol:
        return X_new, f_new
      else:
        X = X_new
        f = f_new
        gradf = gradf_new
        iters = iters + 1
        print("iter= ", iters, "x = ", X[0], "y = ", X[1], "f=", f)
    iters = iters + 1
    
  return X, f


# Test Case
def testCGWithRosenBrock():
  X0 = np.array([-4.0,4.0])

  a = 4
  b = 100
  rosenbrockFuncParams = np.array([a,b])
  maxLineSearchIters = 100
  lineSearchTol = 1e-4
  relTol = 1e-12
  absTol = 1e-16

  opt(X0,rosenbrock, rosenbrockFuncParams, maxLineSearchIters, lineSearchTol, absTol, relTol)


# Run test cases
testCGWithRosenBrock()
      

      
      

  
  
  

iter=  0 x =  -4.0 y =  4.0 f= 14464.0
iter=  1 x =  2.1725760772337908 y =  4.714631616431764 f= 3.3424541086780915
iter=  2 x =  2.1721931982225215 y =  4.715017438482135 f= 3.342037687355509
iter=  3 x =  2.325047166058232 y =  5.364249825884022 f= 2.9784772265710013
iter=  4 x =  2.4990079422709788 y =  6.186607994894793 f= 2.5944152077580758
iter=  5 x =  2.6531597686283375 y =  6.971722092137102 f= 2.270071716379317
iter=  6 x =  2.8048362211014966 y =  7.795310079561646 f= 1.9438851400037547
iter=  7 x =  2.9343941868994357 y =  8.542304683974592 f= 1.602887057153557
iter=  8 x =  3.1555845738904793 y =  9.90155935683354 f= 1.0283718401852633
iter=  9 x =  3.3968744631776913 y =  11.503137345817645 f= 0.4906301105389965
iter=  10 x =  4.068029302556662 y =  16.54917926463927 f= 0.00463802591694589
iter=  11 x =  4.0680591013571155 y =  16.549187717870076 f= 0.004632727950551997
iter=  12 x =  4.068054229636659 y =  16.549035837405103 f= 0.004631464477302643
iter=  13 x =  3.9954